In [ ]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q datasets
!pip install -q sentencepiece
!pip install -q accelerate
!pip install -q evaluate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.5 MB/s eta 0:00:00


In [ ]:
!unzip complete_dataset.zip

Archive:  complete_dataset.zip
   creating: complete_dataset/
  inflating: complete_dataset/.DS_Store  
  inflating: __MACOSX/complete_dataset/._.DS_Store  
  inflating: complete_dataset/dataset_dict.json  
  inflating: __MACOSX/complete_dataset/._dataset_dict.json  
   creating: complete_dataset/train/
  inflating: __MACOSX/complete_dataset/._train  
   creating: complete_dataset/validation/
  inflating: __MACOSX/complete_dataset/._validation  
  inflating: complete_dataset/train/state.json  
  inflating: __MACOSX/complete_dataset/train/._state.json  
  inflating: complete_dataset/train/dataset_info.json  
  inflating: __MACOSX/complete_dataset/train/._dataset_info.json  
  inflating: complete_dataset/train/data-00000-of-00001.arrow  
  inflating: __MACOSX/complete_dataset/train/._data-00000-of-00001.arrow  
  inflating: complete_dataset/validation/state.json  
  inflating: __MACOSX/complete_dataset/validation/._state.json  
  inflating: complete_dataset/validation/dataset_info.json  

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
MODEL = "uqa/mt5-large-UQA-1.0"#
REPO = "mt5-base-uqa"
EPOCHS = 8
DATASET_PATH = "./complete_dataset"

# 1. Load Dataset

In [ ]:
from datasets import Dataset, load_from_disk

dataset = load_from_disk(DATASET_PATH)
dataset["train"] = dataset["train"]
dataset["validation"] = dataset["validation"]
dataset

DatasetDict({
    train: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 495
    })
    validation: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 124
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Trainer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# pipe = pipeline("text2text-generation", model=MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL).to(device)

# model = AutoModelForSeq2SeqLM.from_pretrained(MODEL, torch_dtype=torch.float16).to(device)
# model.params = model.to_fp16(model.params)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


In [ ]:
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s' % (example['question_urdu'], example['context_urdu'])
    example['target_text'] = '%s' % example['answer_urdu']
    return example

def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], truncation=True, padding="max_length", max_length=512)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], truncation=True, padding="max_length", max_length=128)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
    }

    return encodings

In [ ]:
# !du -sh mt5-base-uqa/checkpoint-186/optimizer.pt

## Ignore the below cells

In [ ]:
train_dataset = dataset["train"].map(add_eos_to_examples)
train_dataset = train_dataset.map(convert_to_features, batched=True)

valid_dataset = dataset["validation"].map(add_eos_to_examples, load_from_cache_file=False)
valid_dataset = valid_dataset.map(convert_to_features, batched=True, load_from_cache_file=False)

columns = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [ ]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

In [ ]:
len(train_dataset), len(valid_dataset)

(495, 124)

# 2. Training

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=REPO,
    num_train_epochs=EPOCHS,
    eval_strategy="steps",
    save_steps=62,
    logging_steps=31,
    # save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
)

<ipython-input-11-dae6a9c74c9c>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
31,0.601500,0.370912
62,0.365700,0.326077
93,0.426700,0.295580
124,0.376700,0.282222
155,0.330500,0.273807
186,0.274200,0.273642
217,0.272700,0.267240
248,0.325300,0.257694
279,0.230300,0.258412
310,0.249600,0.253405


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
'''Step	Training Loss	Validation Loss
31	0.601500	0.370912
62	0.365700	0.326077
93	0.426700	0.295580
124	0.376700	0.282222
155	0.330500	0.273807
186	0.274200	0.273642
217	0.272700	0.267240
248	0.325300	0.257694
279	0.230300	0.258412
310	0.249600	0.253405
341	0.243500	0.249812
372	0.279100	0.249070'''

'\nStep\tTraining Loss\tValidation Loss\n31\t1.211600\t0.762529\n62\t0.899800\t0.648082\n93\t0.716700\t0.580091\n124\t0.622500\t0.564062\n155\t0.523500\t0.542392\n186\t0.549400\t0.530387\n217\t0.497400\t0.531491\n248\t0.427100\t0.532043\n279\t0.396400\t0.529665\n310\t0.409200\t0.519797\n341\t0.356700\t0.522532\n372\t0.355800\t0.516591\n403\t0.292600\t0.528990\n434\t0.361700\t0.530401\n465\t0.303200\t0.525231\n496\t0.297800\t0.534052\n527\t0.281900\t0.538114\n558\t0.299800\t0.533996\n589\t0.294300\t0.534162\n620\t0.268400\t0.533698\n'

## Upload to Huggingface

In [ ]:
from huggingface_hub import login

login()


In [ ]:
!sudo apt-get install git-lfs
!git lfs install

In [ ]:
%cd mt5-large-urdu-law-qa

In [ ]:
!ls

In [ ]:
!git lfs install

In [ ]:
!huggingface-cli lfs-enable-largefiles .

In [ ]:
!git lfs track "model.safetensors" "optimizer.pt" "tokenizer.json"

In [ ]:
!git status

In [ ]:
!git add .
!git commit -m "fine tuned mT5-large uploaded with lfs update!"
!git push

# 3. Evaluation

In [ ]:
from tqdm import tqdm

In [ ]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
from glob import glob
from tqdm import tqdm

def evaluate(eval_dataset):
    checkpoints = glob(f"./checkpoint-*")
    predictions = []
    references = []

    print(eval_dataset[0])

    # Iterate through the dataset with index-based access
    for idx in tqdm(range(len(eval_dataset))):
        data = eval_dataset[idx]

        # Tokenize the input text
        input_ids = tokenizer(data["input_text"], return_tensors="pt").input_ids
        # Generate model output
        outputs = model.generate(input_ids.to(device), max_new_tokens=200)
        # Decode the output to text
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append predictions with the current index as ID
        predictions.append({"id": str(idx), "prediction_text": pred})

        # Append references (target_text as the correct answer)
        references.append({"id": str(idx), "answers": [{"text": data["target_text"], "answer_start": 0}]})

    return predictions, references


In [ ]:
val_dataset = dataset["validation"].map(add_eos_to_examples, load_from_cache_file=False)
preds, res = evaluate(eval_dataset=val_dataset)

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

100%|██████████| 124/124 [04:25<00:00,  2.14s/it]


In [ ]:
for i in range(5):
    print(preds[i])
    print(res[i], "\n")

In [ ]:
# Compute the evaluation metric
result = metric.compute(predictions=preds, references=res)
print(result)

Without fine tuning: {'exact_match': 0.0, 'f1': 35.59722983721601}

In [ ]:
import pandas as pd

df = pd.DataFrame(res)
df

,id,answers
0,0,[{'text': 'اسلامی کونسل کا کوئی رکن تین سال کی...
1,1,[{'text': 'پاکستان میں یوم آغاز سے قبل قبائلی ...
2,2,[{'text': 'نہیں، کسی عدالت عالیہ کے جج کا اس ک...
3,3,[{'text': 'وفاقی حکومت یا، جیسی بھی صورت ہو، ص...
4,4,[{'text': 'کسی رکن کو منحرف قرار دینے سے پہلے،...
...,...,...
119,119,[{'text': 'وزیر اعظم مشترکہ مفادات کی کونسل کا...
120,120,[{'text': 'نہیں، کسی شخص کو شہادت حاصل کرنے کی...
121,121,[{'text': 'سینیٹ میں اسپیکر اور ڈپٹی اسپیکر کے...
122,122,[{'text': 'مالی بل کی ابتدا قومی اسمبلی میں ہو...


In [ ]:
df2 = pd.DataFrame(preds)
df2

,id,prediction_text
0,0,اسلامی کونسل کے رکن کی مدت کار تین سال کی مدت ...
1,1,پاکستان میں یوم آغاز سے عین قبل، قبائلی علاقے ...
2,2,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی س...
3,3,وفاقی حکومت یا صوبائی حکومت وقتاً فوقتاً ترمیم...
4,4,پارٹی کے سربراہ کو کسی رکن کو منحرف قرار دینے ...
...,...,...
119,119,مشترکہ مفادات کی کونسل کا چیئرمین وزیر اعظم ہے۔
120,120,نہیں، کسی شخص کو شہادت حاصل کرنے کے لئے اذیت ن...
121,121,سینیٹ میں اسپیکر اور ڈپٹی اسپیکر کے مساوی عہدے...
122,122,مالی بل اپنی قانون سازی کا عمل قومی اسمبلی میں...


In [ ]:
df3 = pd.merge(df, df2, on='id')
display(df3)

,id,answers,prediction_text
0,0,[{'text': 'اسلامی کونسل کا کوئی رکن تین سال کی...,اسلامی کونسل کے رکن کی مدت کار تین سال کی مدت ...
1,1,[{'text': 'پاکستان میں یوم آغاز سے قبل قبائلی ...,پاکستان میں یوم آغاز سے عین قبل، قبائلی علاقے ...
2,2,[{'text': 'نہیں، کسی عدالت عالیہ کے جج کا اس ک...,نہیں، کسی عدالت عالیہ کے جج کا اس کی رضامندی س...
3,3,[{'text': 'وفاقی حکومت یا، جیسی بھی صورت ہو، ص...,وفاقی حکومت یا صوبائی حکومت وقتاً فوقتاً ترمیم...
4,4,[{'text': 'کسی رکن کو منحرف قرار دینے سے پہلے،...,پارٹی کے سربراہ کو کسی رکن کو منحرف قرار دینے ...
...,...,...,...
119,119,[{'text': 'وزیر اعظم مشترکہ مفادات کی کونسل کا...,مشترکہ مفادات کی کونسل کا چیئرمین وزیر اعظم ہے۔
120,120,[{'text': 'نہیں، کسی شخص کو شہادت حاصل کرنے کی...,نہیں، کسی شخص کو شہادت حاصل کرنے کے لئے اذیت ن...
121,121,[{'text': 'سینیٹ میں اسپیکر اور ڈپٹی اسپیکر کے...,سینیٹ میں اسپیکر اور ڈپٹی اسپیکر کے مساوی عہدے...
122,122,[{'text': 'مالی بل کی ابتدا قومی اسمبلی میں ہو...,مالی بل اپنی قانون سازی کا عمل قومی اسمبلی میں...


In [ ]:
df3.to_csv("./mt5-large-responses.csv", index=False)

In [ ]:
!ls